In [17]:
pip install resnet

  Created wheel for resnet: filename=resnet-0.1-py3-none-any.whl size=10041 sha256=021bc90e2834fc483c7d6a3a7c45997bcc4eb518b0f4bfc9dfefb625774c9628
  Stored in directory: c:\users\kiran\appdata\local\pip\cache\wheels\33\b0\d5\b61fef326e57762473e1b108f1bb275537756d4a6bad1fb09d
Successfully built resnet
Note: you may need to restart the kernel to use updated packages.


In [18]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
# Image Data Generator is used for Data augumentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [27]:
# resize all images to this
IMAGE_SIZE = [224,224]
train_path = 'C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Train'
valid_path = 'C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Test'


**Import resnet50 library as shown**

In [79]:

#[3] is for RGB channel
# weights = 'imagenet' means that we should use the weights of imagenet or images of imagenet
#include_top = in resnet there are 1000 o/p categories, but here we have only 3 catgories .hence include_Top = false means first and last layers will be deleted

resnet = ResNet50(input_shape = tuple(IMAGE_SIZE + [3]), weights = 'imagenet', include_top = 'False')

In [31]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [80]:
# dont train the existing weights.This is important because if we dont mention that the existing VGGnet weights will change during training process
for layer in resnet.layers:
    layer.trainable = False

In [39]:
# Glob function will deternime how many folders are there in the given path, which inturn will determine how many outputs are there
folders = glob(train_path+'\*')

In [40]:
folders

['C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Train\\audi',
 'C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Train\\lamborghini',
 'C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Train\\mercedes']

In [43]:
# length of folders will deterine number of outputs in the NN
len(folders)

3

In [45]:
#After downloading the layers, then we have to flatten output layer
x = Flatten()(resnet.output)

In [46]:
x

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'flatten')>

In [48]:
prediction = Dense(len(folders),activation = 'softmax')(x)
# create a model object
model = Model(inputs = resnet.input,outputs = prediction)

In [50]:
# summary of the model
# here we can see that last layer dense(Dense) has three nodes
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [51]:
# tell the model , what cost ans optimization to use
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [53]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Dont do Image augumentation for test Dataset
test_datagen = ImageDataGenerator(rescale = 1./255)

In [54]:
# make sure you provide the same target size as initialized for image size

training_set = train_datagen.flow_from_directory(train_path,
                                                target_size = (224,224),
                                                batch_size = 32,
                                                class_mode = 'categorical')

Found 64 images belonging to 3 classes.


In [57]:
test_set = train_datagen.flow_from_directory(valid_path,
                                                target_size = (224,224),
                                                batch_size = 32,
                                                class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [58]:
# fit the model
# Run the cell

r = model.fit_generator(training_set,
                       validation_data = test_set,
                       epochs = 50,
                       steps_per_epoch = len(training_set),
                       validation_steps = len(test_set)
                       )

C:\Users\kiran\AppData\Local\Temp/ipykernel_11368/1858839874.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(training_set,


Epoch 1/50
2/2 [==============================] - 57s 38s/step - loss: 1.0993 - accuracy: 0.2969 - val_loss: 1.0968 - val_accuracy: 0.5172
Epoch 2/50
2/2 [==============================] - 40s 28s/step - loss: 1.0990 - accuracy: 0.2969 - val_loss: 1.0969 - val_accuracy: 0.5172
Epoch 3/50
2/2 [==============================] - 40s 29s/step - loss: 1.0988 - accuracy: 0.3281 - val_loss: 1.0969 - val_accuracy: 0.5345
Epoch 4/50
2/2 [==============================] - 39s 29s/step - loss: 1.0984 - accuracy: 0.2969 - val_loss: 1.0971 - val_accuracy: 0.2759
Epoch 5/50
2/2 [==============================] - 41s 30s/step - loss: 1.0984 - accuracy: 0.3281 - val_loss: 1.0973 - val_accuracy: 0.3276
Epoch 6/50
2/2 [==============================] - 43s 31s/step - loss: 1.0980 - accuracy: 0.3906 - val_loss: 1.0975 - val_accuracy: 0.3276
Epoch 7/50
2/2 [==============================] - 66s 50s/step - loss: 1.0979 - accuracy: 0.3906 - val_loss: 1.0977 - val_accuracy: 0.3276
Epoch 8/50
2/2 [===========

KeyboardInterrupt: 

In [59]:
y_predict = model.predict(test_set)

In [61]:
# it denotes the probability of three classes
y_predict

array([[0.32377386, 0.31822383, 0.35800233],
       [0.3235135 , 0.31810927, 0.3583772 ],
       [0.323491  , 0.31791845, 0.35859057],
       [0.32380486, 0.3175343 , 0.35866085],
       [0.3236171 , 0.31852284, 0.35786015],
       [0.32379878, 0.3182127 , 0.35798854],
       [0.32352874, 0.31775174, 0.35871953],
       [0.32421768, 0.31823048, 0.35755187],
       [0.3237176 , 0.3178382 , 0.35844424],
       [0.32341743, 0.31810912, 0.35847342],
       [0.32389024, 0.31818026, 0.35792956],
       [0.32379958, 0.31825817, 0.35794225],
       [0.32380474, 0.31813622, 0.35805905],
       [0.323911  , 0.3181111 , 0.35797784],
       [0.32377607, 0.31795833, 0.35826564],
       [0.32388845, 0.3182179 , 0.3578937 ],
       [0.32360598, 0.31805947, 0.35833457],
       [0.32379475, 0.31837597, 0.3578293 ],
       [0.32378805, 0.31811014, 0.35810181],
       [0.3235101 , 0.31825832, 0.35823157],
       [0.32325616, 0.317661  , 0.35908294],
       [0.32323843, 0.31793   , 0.35883152],
       [0.

In [62]:
y_predict.shape

(58, 3)

In [63]:
# How to read the test image and predict from model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [65]:
from tensorflow.keras.models import load_model
model.save('model_resnet50.h5')

C:\Users\kiran\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [66]:
model = load_model('model_resnet50.h5')

**Read the image from the folder**

In [68]:
img = image.load_img('C:/Users/kiran/Desktop/Bhaskar/DataScience_Projects/ml_projects/KN/Car_brand_classification/Datasets/Test/lamborghini/11.jpg',target_size = (224,224))

**Convert image to array**

In [69]:
x = image.img_to_array(img)

In [70]:
x

array([[[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [196., 187., 172.],
        [217., 208., 193.],
        [243., 234., 219.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [245., 245., 237.],
        [243., 243., 235.],
        [242., 242., 234.]],

       [[252., 252., 252.],
        [252., 252., 252.],
        [252., 252., 252.],
        ...,
        [240., 249., 248.],
        [242., 251., 250.],
        [242., 251., 250.]],

       ...,

       [[189., 207., 229.],
        [190., 206., 229.],
        [190., 206., 229.],
        ...,
        [171., 180., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[185., 206., 227.],
        [185., 206., 227.],
        [185., 206., 227.],
        ...,
        [171., 180., 187.],
        [171., 180., 187.],
        [171., 180., 187.]],

       [[185., 206., 227.],
        [185., 206., 227.],
        [185., 2

In [71]:
x.shape

(224, 224, 3)

In [72]:
x = x/255

In [73]:
x

array([[[0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        ...,
        [0.76862746, 0.73333335, 0.6745098 ],
        [0.8509804 , 0.8156863 , 0.75686276],
        [0.9529412 , 0.91764706, 0.85882354]],

       [[0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        ...,
        [0.9607843 , 0.9607843 , 0.92941177],
        [0.9529412 , 0.9529412 , 0.92156863],
        [0.9490196 , 0.9490196 , 0.91764706]],

       [[0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        [0.9882353 , 0.9882353 , 0.9882353 ],
        ...,
        [0.9411765 , 0.9764706 , 0.972549  ],
        [0.9490196 , 0.9843137 , 0.98039216],
        [0.9490196 , 0.9843137 , 0.98039216]],

       ...,

       [[0.7411765 , 0.8117647 , 0.8980392 ],
        [0.74509805, 0.80784315, 0.8980392 ],
        [0.74509805, 0

In [75]:
x = np.expand_dims(x,axis = 0)
img_data = preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [76]:
model.predict(img_data)

array([[0.31811163, 0.31760648, 0.36428186]], dtype=float32)

In [77]:
a  = np.argmax(model.predict(img_data),axis = 1)

In [78]:
a

array([2], dtype=int64)